In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
import os
import glob
import cv2

In [3]:
def load_dataset(data_dir):
    """
    Load all grayscale .jpg images from a directory and flatten them into column vectors.

    Args:
        data_dir (str): Path to the folder containing .jpg images.

    Returns:
        X (np.ndarray): 2D array of shape (D, N), where D = H * W (flattened image size)
                        and N = number of images.
        labels (List[int]): List of length N with integer labels parsed from the
                            first two characters of each filename.
    """
    X = []
    labels = []
    img_path = glob.glob(os.path.join(data_dir, '*.jpg'))
    for path in img_path:
      img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
      if img is None:
        continue
      flatten_img = img.flatten()
      X.append(flatten_img)
      labels.append(int(os.path.basename(path)[:2]))
    X = np.column_stack(X)
    return X, labels

In [19]:
def compute_pairwise_distances(X, Y, metric='l2'):
    """
    Compute pairwise distances between two sets of vectors using the specified metric.

    Args:
        X (np.ndarray): Array of shape (D, M), representing M query vectors of dimensionality D.
        Y (np.ndarray): Array of shape (D, N), representing N reference vectors of dimensionality D.
        metric (str): Distance metric to use; either 'l2' for Euclidean distance or 'l1' for Manhattan distance.

    Returns:
        np.ndarray: Distance matrix of shape (M, N), where entry (i, j) is the distance between X[:, i] and Y[:, j].

    Raises:
        ValueError: If an unsupported metric string is provided.
    """
    if metric == 'l2':
        # Use Euclidean (L2) distance
        return calc_l2(X, Y)
    elif metric == 'l1':
        # Use Manhattan (L1) distance
        return calc_l1(X, Y)
    else:
        # Reject unknown metrics
        raise ValueError(f"Unknown metric: {metric}")

def calc_l2(X, Y):
    """
    Compute pairwise L2 (Euclidean) distances between two sets of vectors.

    Args:
        X (np.ndarray): Array of shape (D, M), M query vectors.
        Y (np.ndarray): Array of shape (D, N), N reference vectors.

    Returns:
        np.ndarray: Matrix of shape (M, N) where entry (i, j) is
                    ||X[:, i] - Y[:, j]||_2.
    """
    diff = X.T[:, np.newaxis, :] - Y.T[np.newaxis, :, :]
    dists = np.sqrt(np.sum(diff ** 2, axis=2))
    return dists


def calc_l1(X, Y):
    """
    Compute pairwise L1 (Manhattan) distances between two sets of vectors.

    Args:
        X (np.ndarray): Array of shape (D, M), M query vectors.
        Y (np.ndarray): Array of shape (D, N), N reference vectors.

    Returns:
        np.ndarray: Matrix of shape (M, N) where entry (i, j) is
                    ||X[:, i] - Y[:, j]||_1.
    """
    M = X.shape[1]
    N = Y.shape[1]
    diffs = np.abs(X[:, :, np.newaxis] - Y[:, np.newaxis, :])
    dists = np.sum(diffs, axis=0)
    return dists

In [20]:
def match_accuracy(X_train, y_train, X_test, y_test, metric='l2'):
    """
    Perform nearest-neighbor classification and compute accuracy.

    For each test vector, find the closest training vector using the specified
    distance metric, then compare its label to the true test label.

    Args:
        X_train (np.ndarray): Training data of shape (D, N_train).
        y_train (List[int]): Labels for training data, length N_train.
        X_test (np.ndarray): Test data of shape (D, N_test).
        y_test (List[int]): True labels for test data, length N_test.
        metric (str): Distance metric to use; either 'l1' or 'l2'.

    Returns:
        float: Classification accuracy (between 0 and 1).
    """
    dist = compute_pairwise_distances(X_test, X_train, metric)
    min_idx = np.argmin(dist, axis=1)
    y_pred = np.array(y_train)[min_idx]
    accuracy = np.mean(np.array(y_pred) == np.array(y_test))

    return accuracy

In [21]:
def proj1(data_root):
    # train: 50 subjects with 7 images each; test/val: 50 subjects with 1 image each
    X_tr, y_tr = load_dataset(os.path.join(data_root, 'train'))
    X_te, y_te = load_dataset(os.path.join(data_root, 'test'))

    for metric in ('l1', 'l2'):
        acc = match_accuracy(X_tr, y_tr, X_te, y_te, metric)
        print(f"proj1 – {metric.upper()} accuracy: {acc * 100:.2f}%")

In [22]:
data_root = '/content/drive/MyDrive/HW1/'
print("===== Project 1 =====")
proj1(data_root)

===== Project 1 =====
proj1 – L1 accuracy: 34.00%
proj1 – L2 accuracy: 98.00%
